In [1]:
from BinanceTrade import *
from binance.client import Client
import pandas as pd
import numpy as np
import requests
import datetime
import yaml

In [2]:
with open('/Users/ho/Documents/config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']

client = Client(api_key=APP_KEY, api_secret=APP_SECRET)

In [3]:
ticker = "BTCUSDT"
interval='1d'   # 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

balance = 1000.0

df = get_ohlcv(client)

In [6]:
df['Cross'] = ma_cross(df)
df

,Time,Open,High,Low,Close,Volume,Cross
0,2020-12-21,23455.54,24102.77,21815.00,22719.71,88030.297243,N
1,2020-12-22,22719.88,23837.10,22353.40,23810.79,87033.126160,N
2,2020-12-23,23810.79,24100.00,22600.00,23232.76,119047.259733,N
3,2020-12-24,23232.39,23794.43,22703.42,23729.20,69013.834252,N
4,2020-12-25,23728.99,24789.86,23433.60,24712.47,79519.943569,N
...,...,...,...,...,...,...,...
995,2023-09-12,25162.53,26567.00,25131.48,25840.10,56434.385370,N
996,2023-09-13,25840.10,26405.22,25764.17,26222.00,31610.827530,N
997,2023-09-14,26222.00,26860.49,26126.77,26522.73,38333.172500,N
998,2023-09-15,26522.73,26888.00,26224.00,26600.00,26227.293690,N


In [5]:
df['MA20'] = get_ma(df, 20)
df['MA120'] = get_ma(df, 120)
df[['MACD','MACD_signal']] = get_macd(df)[['MACD','MACD_signal']]

df['Cross'] = 'N'
for i in range(1,len(df)):
    p = df.index[i-1]
    c = df.index[i]
    if df.loc[c, 'MA20'] == None:
        continue
    elif df.loc[p,'MA20'] < df.loc[p,'MA120'] and df.loc[c,'MA20'] > df.loc[c,'MA120']:
        df.loc[c, 'Cross'] = 'G'
    elif df.loc[p,'MA20'] > df.loc[p,'MA120'] and df.loc[c,'MA20'] < df.loc[c,'MA120']:
        df.loc[c, 'Cross'] = 'D'
# 결과 출력
print(df)

          Time      Open      High       Low     Close         Volume  \
0   2020-12-21  23455.54  24102.77  21815.00  22719.71   88030.297243   
1   2020-12-22  22719.88  23837.10  22353.40  23810.79   87033.126160   
2   2020-12-23  23810.79  24100.00  22600.00  23232.76  119047.259733   
3   2020-12-24  23232.39  23794.43  22703.42  23729.20   69013.834252   
4   2020-12-25  23728.99  24789.86  23433.60  24712.47   79519.943569   
..         ...       ...       ...       ...       ...            ...   
995 2023-09-12  25162.53  26567.00  25131.48  25840.10   56434.385370   
996 2023-09-13  25840.10  26405.22  25764.17  26222.00   31610.827530   
997 2023-09-14  26222.00  26860.49  26126.77  26522.73   38333.172500   
998 2023-09-15  26522.73  26888.00  26224.00  26600.00   26227.293690   
999 2023-09-16  26599.99  26777.00  26445.00  26566.93   11570.996580   

           MA20         MA120        MACD  MACD_signal Cross  
0           NaN           NaN    0.000000     0.000000     N

In [6]:
df['Cross'].unique()

array(['N', 'D', 'G'], dtype=object)

In [19]:
# 레버리지 변경
client.futures_change_leverage(
    symbol="BTCUSDT",
    leverage=3,
)

{'symbol': 'BTCUSDT', 'leverage': 3, 'maxNotionalValue': '200000000'}

In [20]:
# 선물 마진 타입 변경
client.futures_change_margin_type(
    symbol="BTCUSDT",
    marginType="ISOLATED",  # ISOLATED, CROSSED
)

BinanceAPIException: APIError(code=-4046): No need to change margin type.

In [ ]:
# 선물 잔고 조회
account = client.futures_account()
for i in account['assets']:
    if i['asset'] == 'USDT':
        print(i)
for i in account['positions']:
    if i['symbol'] == 'BTCUSDT':
        print(i)

{'asset': 'USDT', 'walletBalance': '0.00000000', 'unrealizedProfit': '0.00000000', 'marginBalance': '0.00000000', 'maintMargin': '0.00000000', 'initialMargin': '0.00000000', 'positionInitialMargin': '0.00000000', 'openOrderInitialMargin': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'marginAvailable': True, 'updateTime': 0}
{'symbol': 'BTCUSDT', 'initialMargin': '0', 'maintMargin': '0', 'unrealizedProfit': '0.00000000', 'positionInitialMargin': '0', 'openOrderInitialMargin': '0', 'leverage': '3', 'isolated': True, 'entryPrice': '0.0', 'breakEvenPrice': '0.0', 'maxNotional': '200000000', 'positionSide': 'BOTH', 'positionAmt': '0.000', 'notional': '0', 'isolatedWallet': '0', 'updateTime': 1693184330937, 'bidNotional': '0', 'askNotional': '0'}


In [ ]:
# 주문 생성
try:
    client.futures_create_order(
        symbol="BTCUSDT",  # 주문 종목
        timeInForce='GTC',
        type="LIMIT",   # LIMIT, MARKET
        side='BUY',     # BUY, SELL
        price=20000,     # 주문 희망가
        quantity=1     # 주문 수량
    )
except Exception as e:
    print(e)

APIError(code=-2019): Margin is insufficient.


In [ ]:
# 모든 주문 취소
client.futures_cancel_all_open_orders(
    symbol="BTCUSDT"
)

{'code': 200, 'msg': 'The operation of cancel all open order is done.'}

In [20]:
def heikinashi_upswing_strategy(df,s=13,l=21):
    """하이킨아시 업스윙 전략"""
    x = get_heikinashi(df)
    x['RSI'] = get_rsi(df)
    x['Cross'] = ma_cross(x,s,l)
    x['Action'] = 'Stay'
    for i in range(len(df)):
        t = x.index[i]
        if x.loc[t, 'UD']=='Up' and x.loc[t, 'Cross']=='G':
            x.loc[t,'Action'] = 'Buy'
        elif x.loc[t, 'UD']=='Down' and x.loc[t, 'Cross']=='D' and x.loc[t, 'RSI']>30:
            x.loc[t,'Action'] = 'Sell'
    
    return x

In [21]:
hs = heikinashi_upswing_strategy(df)
print(len(hs[hs['Action'] == 'Buy']))
print(len(hs[hs['Action'] == 'Sell']))

17
20


In [33]:
balance = 1000.0
fee = 0.0
rate = 0.3
hs = heikinashi_upswing_strategy(df)
hs[['Balance', 'Coin Held', 'Quantity Held', 'Average Price', 'ROR']] = 0.0
hs['Average Price'] = 0.0
for i in range(1,len(hs)):
    if hs.loc[i, 'Action'] == 'Buy':
        last_balance = hs.loc[i-1, 'Balance']
        buy_price = df.loc[i, 'Close']
        hs.loc[i, 'Quantity Held'] = hs.loc[i-1, 'Quantity Held'] + (last_balance * (1-rate)) / buy_price
        hs.loc[i, 'Average Price'] = (hs.loc[i-1, 'Coin Held'] + last_balance * (1-rate)) / hs.loc[i, 'Quantity Held']
        hs.loc[i, 'Coin Held'] = hs.loc[i, 'Average Price'] * hs.loc[i, 'Quantity Held']
        hs.loc[i, 'Balance'] = last_balance * (1-rate)
hs

/var/folders/5n/y13cbnyn1t1352k1d37bjkym0000gn/T/ipykernel_32103/3812124898.py:12: RuntimeWarning: invalid value encountered in double_scalars
  hs.loc[i, 'Average Price'] = (hs.loc[i-1, 'Coin Held'] + last_balance * (1-rate)) / hs.loc[i, 'Quantity Held']


,Close,Open,High,Low,UD,RSI,Cross,Action,Balance,Coin Held,Quantity Held,Average Price,ROR
0,23023.2550,23087.625000,24102.77,21815.000000,Down,NaN,N,Stay,0.0,0.0,0.0,0.0,0.0
1,23180.2925,23055.440000,23837.10,22353.400000,Up,NaN,N,Stay,0.0,0.0,0.0,0.0,0.0
2,23435.8875,23117.866250,24100.00,22600.000000,Up,NaN,N,Stay,0.0,0.0,0.0,0.0,0.0
3,23364.8600,23276.876875,23794.43,22703.420000,Up,NaN,N,Stay,0.0,0.0,0.0,0.0,0.0
4,24166.2300,23320.868438,24789.86,23320.868438,Up,NaN,N,Stay,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,25675.2775,25669.393105,26567.00,25131.480000,Up,42.780299,N,Stay,0.0,0.0,0.0,0.0,0.0
996,26057.8725,25672.335303,26405.22,25672.335303,Up,47.503164,N,Stay,0.0,0.0,0.0,0.0,0.0
997,26432.9975,25865.103901,26860.49,25865.103901,Up,50.937343,N,Stay,0.0,0.0,0.0,0.0,0.0
998,26558.6825,26149.050701,26888.00,26149.050701,Up,51.809648,N,Stay,0.0,0.0,0.0,0.0,0.0


In [38]:
balance = 1000.0
fee = 0.0
rate = 0.3
hs = heikinashi_upswing_strategy(df)
hs[['Balance', 'Coin Held', 'Quantity']] = 0.0
quantity_held = 0
coin_held = 0
hs.loc[0, 'Balance'] = balance
for i in range(1,len(hs)):
    if hs.loc[i, 'Action'] == 'Buy':
        buy_price = df.loc[i, 'Close']
        hs.loc[i, 'Quantity'] = quantity_held + (balance * (1-rate)) / buy_price
        hs.loc[i, 'Coin Held'] = coin_held + balance * (1-rate)
        hs.loc[i, 'Balance'] = balance - balance * (1-rate)
    elif hs.loc[i, 'Action'] == 'Sell':
        sell_price = df.loc[i, 'Close']
        hs.loc[i, 'Quantity'] = quantity_held - (coin_held * (1-rate)) / sell_price
        hs.loc[i, 'Coin Held'] = coin_held - coin_held * (1-rate)
        hs.loc[i, 'Balance'] = balance + coin_held * (1-rate)
hs

,Close,Open,High,Low,UD,RSI,Cross,Action,Balance,Coin Held,Quantity
0,23023.2550,23087.625000,24102.77,21815.000000,Down,NaN,N,Stay,0.0,0.0,0.0
1,23180.2925,23055.440000,23837.10,22353.400000,Up,NaN,N,Stay,0.0,0.0,0.0
2,23435.8875,23117.866250,24100.00,22600.000000,Up,NaN,N,Stay,0.0,0.0,0.0
3,23364.8600,23276.876875,23794.43,22703.420000,Up,NaN,N,Stay,0.0,0.0,0.0
4,24166.2300,23320.868438,24789.86,23320.868438,Up,NaN,N,Stay,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
995,25675.2775,25669.393105,26567.00,25131.480000,Up,42.780299,N,Stay,0.0,0.0,0.0
996,26057.8725,25672.335303,26405.22,25672.335303,Up,47.503164,N,Stay,0.0,0.0,0.0
997,26432.9975,25865.103901,26860.49,25865.103901,Up,50.937343,N,Stay,0.0,0.0,0.0
998,26558.6825,26149.050701,26888.00,26149.050701,Up,51.809648,N,Stay,0.0,0.0,0.0


In [34]:
hs[hs['Action'] == 'Buy']

,Close,Open,High,Low,UD,RSI,Cross,Action,Balance,Coin Held,Quantity Held,Average Price,ROR
47,39161.9275,36923.128463,40955.51,36923.128463,Up,64.568754,G,Buy,0.0,NaN,0.0,NaN,0.0
82,59083.0850,56028.502736,61844.00,56028.502736,Up,72.710810,G,Buy,0.0,NaN,0.0,NaN,0.0
106,58507.3500,58159.586012,59495.24,57413.020000,Up,55.730244,G,Buy,0.0,NaN,0.0,NaN,0.0
137,56900.0150,56290.343471,58650.00,55241.630000,Up,52.926348,G,Buy,0.0,NaN,0.0,NaN,0.0
219,39787.0425,36827.696215,40900.00,36827.696215,Up,70.402550,G,Buy,0.0,NaN,0.0,NaN,0.0
290,54447.3825,51150.693059,55332.31,51150.693059,Up,65.804378,G,Buy,0.0,NaN,0.0,NaN,0.0
323,67305.0325,63660.109344,68524.25,63660.109344,Up,65.746463,G,Buy,0.0,NaN,0.0,NaN,0.0
413,43091.8025,40910.857210,44500.50,40910.857210,Up,62.911304,G,Buy,0.0,NaN,0.0,NaN,0.0
457,42506.0800,41598.731974,43025.96,41598.731974,Up,58.065454,G,Buy,0.0,NaN,0.0,NaN,0.0
571,20750.2925,20155.910122,21200.00,20155.910122,Up,46.259934,G,Buy,0.0,NaN,0.0,NaN,0.0
